# Tutorial 2: Features & Visibility

**Goal:** Create FeatureProviders that define **what** agents observe and **who** can see it.

**Time:** ~10 minutes

---

## The Key Idea: Declarative Visibility

In traditional frameworks, you manually filter observations:
```python
# Manual filtering (tedious, error-prone)
def get_battery_obs(global_state, agent_id):
    if agent_id == 'battery_1':
        return global_state['battery_1']['soc']  # Owner sees own state
    elif agent_id == 'microgrid_1':
        return global_state['battery_1']['soc']  # Coordinator sees subordinate
    else:
        return None  # Others can't see
```

With HERON, visibility is **declared**, not coded:
```python
class BatterySOC(FeatureProvider):
    visibility = ['owner', 'upper_level']  # That's it!
    soc: float = 0.5
```

### Visibility Levels

| Level | Who Can See | Example Use |
|-------|------------|-------------|
| `public` | Everyone | Time of day, market prices |
| `owner` | The owning agent | Internal device state |
| `upper_level` | Owner + parent in hierarchy | Battery SOC visible to microgrid |
| `system` | System operator (top level) | Grid-wide metrics |

## The FeatureProvider Interface

Every feature implements these methods:

```python
@dataclass
class MyFeature(FeatureProvider):
    visibility = ['owner', 'upper_level']  # Who can see this
    
    value: float = 0.0  # Your data
    
    def vector(self) -> np.ndarray:       # → ML observation
        return np.array([self.value])
    
    def names(self) -> List[str]:         # → Debug/logging
        return ['value']
    
    def to_dict(self) -> Dict:            # → Serialization
        return {'value': self.value}
    
    @classmethod
    def from_dict(cls, d) -> 'MyFeature': # → Deserialization
        return cls(**d)
    
    def set_values(self, **kwargs):       # → Update values
        if 'value' in kwargs:
            self.value = kwargs['value']
```

That's the full interface. Now let's build some real features.

## Example 1: Battery State of Charge

**Visibility:** Owner (battery) + upper_level (microgrid coordinator)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List
import numpy as np

from heron.core.feature import FeatureProvider


@dataclass
class BatterySOC(FeatureProvider):
    """Battery state of charge. Visible to owner and coordinator."""
    visibility = ['owner', 'upper_level']
    
    soc: float = 0.5  # State of charge [0, 1]
    
    def vector(self) -> np.ndarray:
        return np.array([self.soc], dtype=np.float32)
    
    def names(self) -> List[str]:
        return ['soc']
    
    def to_dict(self) -> Dict:
        return {'soc': self.soc}
    
    @classmethod
    def from_dict(cls, d) -> 'BatterySOC':
        return cls(**d)
    
    def set_values(self, **kw):
        if 'soc' in kw:
            self.soc = np.clip(float(kw['soc']), 0.0, 1.0)


# Test it
battery_soc = BatterySOC(soc=0.75)
print(f"Vector: {battery_soc.vector()}")
print(f"Visibility: {battery_soc.visibility}")

## Example 2: Generator Output

**Visibility:** Owner + upper_level + system (grid operator needs to see total generation)

In [ ]:
@dataclass
class GenOutput(FeatureProvider):
    """Generator power output. Visible to owner, coordinator, AND system."""
    visibility = ['owner', 'upper_level', 'system']
    
    p_mw: float = 0.0   # Power output [MW]
    p_max: float = 5.0  # Maximum capacity [MW]
    
    def vector(self) -> np.ndarray:
        # Normalize to [0, 1] for better ML training
        p_norm = self.p_mw / max(self.p_max, 1e-6)
        return np.array([p_norm], dtype=np.float32)
    
    def names(self) -> List[str]:
        return ['p_normalized']
    
    def to_dict(self) -> Dict:
        return {'p_mw': self.p_mw, 'p_max': self.p_max}
    
    @classmethod
    def from_dict(cls, d) -> 'GenOutput':
        return cls(**d)
    
    def set_values(self, **kw):
        if 'p_mw' in kw:
            self.p_mw = float(kw['p_mw'])


# Test it
gen = GenOutput(p_mw=2.5, p_max=5.0)
print(f"Vector (normalized): {gen.vector()}")  # [0.5]
print(f"Raw p_mw: {gen.p_mw} MW")

## Example 3: Public Information

**Visibility:** `public` — everyone sees this (time of day, market prices)

In [ ]:
@dataclass
class TimeOfDay(FeatureProvider):
    """Time feature - visible to everyone."""
    visibility = ['public']
    
    hour: int = 0  # 0-23
    
    def vector(self) -> np.ndarray:
        # Encode cyclically (hour 23 is close to hour 0)
        rad = 2 * np.pi * self.hour / 24
        return np.array([np.sin(rad), np.cos(rad)], dtype=np.float32)
    
    def names(self): return ['hour_sin', 'hour_cos']
    def to_dict(self): return {'hour': self.hour}
    @classmethod
    def from_dict(cls, d): return cls(hour=d['hour'])
    def set_values(self, **kw):
        if 'hour' in kw: self.hour = int(kw['hour']) % 24


# Test
time = TimeOfDay(hour=14)  # 2 PM
print(f"Time vector: {time.vector()}")
print(f"Visibility: {time.visibility}")

## How Visibility Filtering Works

HERON automatically checks visibility based on agent hierarchy:

In [ ]:
# BatterySOC has visibility = ['owner', 'upper_level']
battery_soc = BatterySOC(soc=0.8)

# Check who can see this feature:

# 1. Owner (the battery itself) - YES
can_owner_see = battery_soc.is_observable_by(
    requestor_id='battery_1', requestor_level=1,
    owner_id='battery_1', owner_level=1
)
print(f"Battery can see its own SOC: {can_owner_see}")  # True

# 2. Coordinator (upper_level) - YES  
can_coord_see = battery_soc.is_observable_by(
    requestor_id='microgrid_1', requestor_level=2,  # One level above
    owner_id='battery_1', owner_level=1
)
print(f"Coordinator can see battery SOC: {can_coord_see}")  # True

# 3. Another battery (peer, not upper_level) - NO
can_peer_see = battery_soc.is_observable_by(
    requestor_id='battery_2', requestor_level=1,  # Same level, not owner
    owner_id='battery_1', owner_level=1
)
print(f"Other battery can see SOC: {can_peer_see}")  # False

## Composing Agent State from Features

`FieldAgentState` collects features and handles visibility filtering automatically:

In [ ]:
from heron.core.state import FieldAgentState

# Create state for a battery agent
battery_state = FieldAgentState(owner_id='battery_1', owner_level=1)

# Register features
battery_state.register_feature('soc', BatterySOC(soc=0.7))

# Get full observation vector (all features)
full_obs = battery_state.vector()
print(f"Full state vector: {full_obs}")

# Get observation filtered by requestor
# (In practice, HERON does this automatically based on agent hierarchy)
filtered_obs = battery_state.observed_by(requestor_id='battery_1', requestor_level=1)
print(f"Filtered observation: {filtered_obs}")

## Design Guidelines

| Feature Type | Visibility | Example |
|--------------|------------|---------|
| Private device state | `['owner']` | Internal temperature |
| Hierarchical info | `['owner', 'upper_level']` | Battery SOC, generator output |
| System metrics | `['owner', 'upper_level', 'system']` | Total generation |
| Public info | `['public']` | Time, weather, prices |

**Tips:**
- **Normalize in `vector()`**: ML algorithms prefer [0, 1] or [-1, 1]
- **Store raw values**: Keep originals for debugging
- **Use `@dataclass`**: Clean definitions, auto-generated `__init__`

## Summary: Features We Built

```python
BatterySOC   →  visibility = ['owner', 'upper_level']
GenOutput    →  visibility = ['owner', 'upper_level', 'system']
TimeOfDay    →  visibility = ['public']
```

These features will be used by agents in the next tutorial.

In [ ]:
# Verify our features work
print("Features defined successfully!")
print(f"  BatterySOC: {BatterySOC(soc=0.5).vector()}")
print(f"  GenOutput: {GenOutput(p_mw=2.5, p_max=5.0).vector()}")
print(f"  TimeOfDay: {TimeOfDay(hour=12).vector()}")

## Key Takeaways

1. **Visibility is declarative** — Just set `visibility = ['owner', 'upper_level']`
2. **No manual filtering** — HERON handles observation filtering automatically
3. **Features compose into State** — `FieldAgentState` manages feature collections
4. **Normalize for ML** — Return [0,1] in `vector()`, keep raw values internally

---

**Next:** [03_building_agents.ipynb](03_building_agents.ipynb) — Create agents that use these features